In [11]:
%matplotlib inline

import os
import cmaps
import salem
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
from pylab import rcParams
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.patches as mpatches
from matplotlib.gridspec import GridSpec
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from myfunc import timer
from myfunc import DirMan
import config
import matplotlib.font_manager as fm
from matplotlib.patches import Patch
from matplotlib.colors import ListedColormap, BoundaryNorm
from shapely.geometry import box


# configuration
resolution = "0p1"
# resolution = "500"
# region = [-180,180,-60,90]
region = [-124.8,-66.95,24.5,49.4]
xmin,xmax,ymin,ymax = [-124.8,-66.95,24.5,49.4]
data_path = f'/tera11/zhwei/students/Xionghui/data/run/{resolution}/'
post_data_path = '/tera11/zhwei/students/Xionghui/data/'
shp_path = '/tera11/zhwei/students/Xionghui/data/Shp/'
fig_path = f'/home/xuxh22/stu01/Bedrock/fig/{resolution}/'
path = '/home/xuxh22/stu01/Bedrock/'

if resolution == "0p1":
    size = 0.1
elif resolution == "500":
    size = 0.0005


print('python draw_g2_scatter.py')
print(data_path)
dir_man = DirMan(data_path)
dir_man.enter()
os.makedirs(f'{fig_path}/global_map_2', exist_ok=True)


shp = gpd.GeoDataFrame.from_file(shp_path+'US/USA_adm0.shp')

pd.set_option('display.max_columns', None)
font = {'family': 'Times New Roman'}
matplotlib.rc('font', **font)

params = {'backend': 'ps',
          'axes.labelsize': 25,
          'grid.linewidth': 0.2,
          'font.size': 25,
          'legend.fontsize': 18,
          'legend.frameon': False,
          'xtick.labelsize': 30,
          'xtick.direction': 'out',
          'ytick.labelsize': 30,
          'ytick.direction': 'out',
          'legend.handlelength': 1,
          'legend.handleheight': 1,
          'savefig.bbox': 'tight',
          'axes.unicode_minus': False,
          "mathtext.default":"regular",
          'text.usetex': False}
rcParams.update(params)

font_properties = fm.FontProperties(weight='bold')

python draw_g2_scatter.py
/tera11/zhwei/students/Xionghui/data/run/0p1/
Entered /tera11/zhwei/students/Xionghui/data/run/0p1/


In [12]:
def draw(name,level,cmap):
    image = xr.open_dataset(f'/tera11/zhwei/students/Xionghui/data/US/{name[0]}.nc').sel(lon=slice(region[0],region[1]),lat=slice(region[2],region[3]))
    s = image[f'{name[1]}']
    s = s.salem.roi(shape=shp)

    # if ((name[1]=='LC') or (name[1]=='tp') or (name[1]=='et')):
    #     s = s[0,:,:]

    # if (name[2]=='Sb') or (name[2]=='Sp') or (name[1]=='FD') or (name[1]=='Dr'):
    #     s = np.where(s<=0, np.nan, s)  

    # if (name[2]=='P1'):
    #     s = s*100
    # s = np.where(s==0, np.nan, s)    
    s = np.ma.masked_where(np.isnan(s), s)  

    fig = plt.figure(figsize=(12, 6), dpi=500)

    fig.subplots_adjust(left=0, right=1, 
                    bottom=0, top=1) 
        
    #Create a subgraph grid with 2 rows and 3 columns
    gs = GridSpec(2, 6)
    ax = fig.add_subplot(gs[:, :], projection=ccrs.PlateCarree())

        # Set drawing mode(note:extent's lat from positive to negative)
    img = ax.imshow(s, cmap=cmap,
                    extent=[region[0], region[1], region[3], region[2]],
                    vmin=level[0], vmax=level[-1])

    bbox = gpd.GeoSeries([box(xmin, ymin, xmax, ymax)], crs=shp.crs)
    clipped_shapefile = gpd.clip(shp, bbox)
    clipped_shapefile.plot(ax=ax, facecolor='none', edgecolor='black',linewidth=0.8)

    # shp.boundary.plot(ax=ax, edgecolor='black', linewidth=0.75, alpha=1)

    for spine in ax.spines.values():
        spine.set_edgecolor('black')  
        spine.set_linewidth(0)  

    ax.set_xlim(region[0], region[1])
    ax.set_ylim(region[2], region[3])

    # coastline = cfeature.NaturalEarthFeature('physical', 'coastline', '50m', edgecolor='0.6', facecolor='none')
    rivers = cfeature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', '110m', edgecolor='0.6', facecolor='none')
    ax.add_feature(cfeature.LAND, facecolor='0.95')
    # ax.add_feature(coastline, linewidth=0.6)
    ax.add_feature(cfeature.LAKES, alpha=1, facecolor='white', edgecolor='white')
    ax.add_feature(rivers, linewidth=0.8)
    # ax.gridlines(draw_labels=False, linestyle=':', linewidth=0.7, color='grey', alpha=0.8)

    # ax.add_feature(cfeature.COASTLINE)
    ax.set_extent(region, crs=ccrs.PlateCarree())
    ax.xaxis.set_major_formatter(LongitudeFormatter())
    ax.yaxis.set_major_formatter(LatitudeFormatter())

        # From the bottom left corner x, y, width, height
    # cbar_ax = fig.add_axes([0.4, 0.02, 0.4, 0.03], frameon = False) 
    # cb = fig.colorbar(img, 
    #                 drawedges=False,
    #                 ticks=level, 
    #                 cax=cbar_ax, 
    #                 orientation='horizontal',
    #                 spacing='uniform')

    # cb.ax.tick_params(labelsize=20)
    # cb.ax.yaxis.set_tick_params(direction='out', width=1.5)
    # for label in cb.ax.get_xticklabels() + cb.ax.get_yticklabels():
    #     label.set_fontproperties(font_properties)
    # cb.set_label(f'{name[3]}', fontsize=25, fontweight='bold',labelpad=-117)

    plt.tight_layout()
    plt.show()

In [ ]:
name = ['deficits/Sbedrock', 'Band1', 'Sb', '$S_{{bedrock}}$ (mm)']
level = np.arange(0,500,50)
rgb_list = ['#606060','#8ec0cb','#00CC66','#66CC00',
                                '#69aa4c','#CCCC00','#ebc874','#99004C','#FF6666']
cmap = colors.ListedColormap(rgb_list)
draw(name,level,cmap)

In [ ]:
name = ['deficits/Sr', 'Band1', 'Sr', '$S_{{r}}$ (mm)']
level = level1
cmap = cmap1
draw(name,level,cmap)

In [ ]:
name = ['products_used/gNATSGO/Ssoil_500m', 'Band1', 'Ss', '$S_{{soil}}$ (mm)']
level = level1
cmap = cmap1
draw(name,region,level,cmap)

In [ ]:
name = ['deficits/Sbedrock_dividedby_Sr', 'Band1', 'P1', '$S_{{bedrock}}$/$S_{{r}}$ (%)']
level = level3
cmap = cmaps.StepSeq25_r
draw(name,region,level,cmap)